In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas

## Daten einlesen und Spalten kennenlernen

In [16]:
df = pd.read_csv('../data/immo_data_202208_v2.csv', low_memory=False)

In [17]:
# print all columns, so we can see all columns
pd.set_option('display.max_columns', None)

df.head() 

,Unnamed: 0.1,Unnamed: 0,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,Floor,detail_responsive#municipality,detail_responsive#surface_living,detail_responsive#floor,detail_responsive#available_from,Gemeinde,Wohnfläche,Stockwerk,Nutzfläche,Verfügbarkeit,Grundstücksfläche,detail_responsive#surface_property,detail_responsive#surface_usable,Commune,Surface habitable,Surface du terrain,Surface utile,Disponibilité,Étage,Comune,Superficie abitabile,Disponibilità,Gross return,Piano,Superficie del terreno,Superficie utile,Municipality_merged,Floor_merged,Living_space_merged,Floor_space_merged,Plot_area_merged,Availability_merged,location_parsed,title,details,address,price,link,details_structured,lat,lon,index,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Locality,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,PopulationDensityM,PopulationDensityS,RiversAndLakesL,RiversAndLakesM,RiversAndLakesS,WorkplaceDensityL,WorkplaceDensityM,WorkplaceDensityS,Zip,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_average_house_hold,gde_empty_apartments,gde_foreigners_percentage,gde_new_homes_per_1000,gde_politics_bdp,gde_politics_cvp,gde_politics_evp,gde_politics_fdp,gde_politics_glp,gde_politics_gps,gde_politics_pda,gde_politics_rights,gde_politics_sp,gde_politics_svp,gde_pop_per_km2,gde_population,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,price_cleaned,type,Space extracted,rooms,plz_parsed,type_unified,Floor_unified,Plot_area_unified,Living_area_unified,provider,space,price_s,address_s,No. of rooms:,Number of apartments:,Surface living:,Land area:,Room height:,Last refurbishment:,Year built:,features,description_detailed,Floor space:,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned
0,0,0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,4. floor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Biberstein,4. floor,100 m²,NaN,NaN,On request,Strasse: plz:5023 Stadt: Biberstein Kanton: AG,Luxuriöse Attika-Wohnung mit herrlicher Aussicht,"3.5 rooms, 100 m²,","5023 Biberstein, AG","CHF 1,150,000.—",/en/d/penthouse-buy-biberstein/7255200,"{'Municipality': 'Biberstein', 'Living space':...",47.417100,8.085600,16620,0.511176,0.286451,0.090908,47.415927,Biberstein,8.085840,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,5023,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,1150000.0,penthouse,100.0,5.0,5023.0,penthouse,4,NaN,100.0,Immoscout24.ch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5023.0,NaN,NaN,NaN,NaN
1,1,1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Biberstein,NaN,156 m²,242 m²,222 m²,On request,Strasse:Buhldenstrasse 8d plz:5023 Stadt: Bib...,"Stilvolle Liegenschaft - ruhige Lage, unverbau...","4.5 rooms, 156 m²,","Buhldenstrasse 8d, 5023 Biberstein, AG","CHF 1,420,000.—",/en

### Konvertieren von Datentypen

In [18]:
import ast

def convert_to_dict(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return {}
    elif isinstance(val, dict):
        return val
    else:
        return {}

df['details_structured'] = df['details_structured'].apply(convert_to_dict)

In [19]:
def update_row_values(row):
    details = row['details_structured']
    for key, value in details.items():
        if key in df.columns and pd.isna(row[key]):
            row[key] = value
    return row

df = df.apply(update_row_values, axis=1)


### Merge Columns

In [20]:
def merge_columns(row, main_column, columns_to_merge):
    if pd.isna(row[main_column]):
        for col in columns_to_merge:
            if pd.notna(row[col]):
                row[main_column] = row[col]
                break
    return row

In [21]:
df = df.apply(lambda row: merge_columns(row, 'Living space', ['detail_responsive#surface_living', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile', 'Living_space_merged', 'Surface living:']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Municipality', ['detail_responsive#municipality', 'Gemeinde', 'Commune', 'Comune', 'Municipality_merged']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Availability', ['detail_responsive#available_from', 'Verfügbarkeit', 'Disponibilité', 'Disponibilità', 'Availability_merged']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Floor', ['detail_responsive#floor', 'Étage', 'Piano', 'Floor_merged']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Plot_area_merged', ['detail_responsive#surface_property', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'Plot_area_merged', 'Land area:']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Gross return', ['Gross yield:']), axis=1)

### Clean Columns

In [22]:
def clean_column_values(column):
    # Remove ' m²' and ' m2' substrings
    column = column.str.replace(' m²', '').str.replace(' m2', '')
    # Remove commas
    column = column.str.replace(',', '')
    # Convert to float
    column = column.astype(float)
    return column

columns_to_clean = ['Living_space_merged', 'Plot_area_merged', 'Floor_space_merged']

for col in columns_to_clean:
    df[col] = clean_column_values(df[col])

#### Price

In [23]:
# create new column with price_s as double value 
df['price_search'] = df['price_s'].str.replace('[^\d.]', '', regex=True).astype(float)

# merge price_cleaned to price_search
df['price_search'] = df['price_search'].fillna(df['price_cleaned'])

C:\Users\Oliver\AppData\Local\Temp\ipykernel_13376\3695164647.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['price_search'] = df['price_s'].str.replace('[^\d.]', '', regex=True).astype(float)


In [24]:
numeric_mask = df['price_search'].notna() & df['price_cleaned'].notna()

# Create a boolean mask for rows where the values in the two columns are not the same
not_equal_mask = df['price_search'] != df['price_cleaned']

# Combine the two masks using the & operator
final_mask = numeric_mask & not_equal_mask

# Use the final mask to filter the DataFrame
df[final_mask]["details_structured"]

22477    {}
22478    {}
22480    {}
Name: details_structured, dtype: object

In [25]:
# show details_structured from df[final_mask]
df[final_mask][['details_structured', 'price_search', 'price_cleaned']]

,details_structured,price_search,price_cleaned
22477,{},2194770.0,1490000.0
22478,{},2135850.0,1450000.0
22480,{},1148940.0,780000.0


to drop:

Unnamed: 0.1
Unnamed: 0
Municipality
Living space
Plot area
Floor space
Availability
location
url
detail_responsive#available_from
Floor
Gemeinde
Wohnfläche
Stockwerk
Nutzfläche
Verfügbarkeit
Grundstücksfläche
Commune
Surface habitable
Surface du terrain
Surface utile
Disponibilité
Étage
Comune
Superficie abitabile
Disponibilità
Piano
Superficie del terreno
Superficie utile
address
link
type
Locality

In [26]:
# Define a list of columns to be dropped
columns_to_drop = [
    'Unnamed: 0.1', 'Unnamed: 0', 'Municipality', 'Living space', 'Plot area', 'Floor space',
    'Availability', 'location', 'url', 'detail_responsive#available_from', 'Floor', 'Gemeinde',
    'Wohnfläche', 'Stockwerk', 'Nutzfläche', 'Verfügbarkeit', 'Grundstücksfläche', 'Commune',
    'Surface habitable', 'Surface du terrain', 'Surface utile', 'Disponibilité', 'Étage', 'Comune',
    'Superficie abitabile', 'Disponibilità', 'Piano', 'Superficie del terreno', 'Superficie utile',
    'address', 'link', 'type', 'Locality', 'table', 'description', 'detailed_description', 
]

# Drop the columns from the DataFrame
df = df.drop(columns=columns_to_drop)

In [27]:
# export df to csv
df.to_csv('../data/immo_data_clean.csv', index=False)

# Handling price
 - price_cleaned
 - price_s
 - price
 - description
 - details_structured

In [28]:
# print the number of NaNs in 'price_cleaned'
print(df['price_cleaned'].isnull().sum())
print(df['price_s'].isnull().sum())
print(df['price'].isnull().sum())

1015
13355
0


In [29]:
# Function to extract keys from dictionaries in each cell and append them to a list
keys_list = []
df['details_structured'].apply(lambda x: keys_list.extend(x.keys()) if isinstance(x, dict) else None)

# Remove duplicates if necessary
unique_keys_list = list(set(keys_list))

# Print the list of unique keys
print(unique_keys_list)

['detailed_description', 'Floor', 'Stockwerk', 'Superficie abitabile', 'Gross return', 'Living space', 'Wohnfläche', 'Piano', 'detail_responsive#floor', 'location', 'Grundstücksfläche', 'Availability', 'Étage', 'Floor space', 'Nutzfläche', 'detail_responsive#surface_living', 'detail_responsive#surface_property', 'Gemeinde', 'detail_responsive#surface_usable', 'table', 'Comune', 'description', 'detail_responsive#municipality', 'Disponibilità', 'Municipality', 'url', 'detail_responsive#available_from', 'Surface habitable', 'Superficie utile', 'Superficie del terreno', 'Disponibilité', 'Verfügbarkeit', 'Commune', 'Surface du terrain', 'Surface utile', 'Plot area']


In [30]:
# Filtern Sie alle Zeilen, bei denen 'price' nicht mit 'CHF' beginnt
non_chf_prices = df.loc[~df['price'].str.startswith('CHF'), 'price']

# Konvertieren Sie die gefilterten Preise in eine Liste
non_chf_prices_list = non_chf_prices.unique().tolist()

# Drucken Sie die Liste
print(non_chf_prices_list)

['Price on request', 'EUR 635,000.—', 'EUR 780,000.—', 'EUR 495,000.—', 'EUR 190,000.—', 'EUR 650,000.—', 'EUR 1,639,000.—', 'EUR1,130,000.–', 'EUR1,200,000.–', 'EUR1,340,000.–', 'EUR1,645,000.–', 'EUR1,880,000.–', 'EUR2,130,000.–', 'EUR5,500,000.–', 'EUR10,000,000.–', 'EUR200,000.–', 'EUR1,180,000.–']


In [31]:
# print the nomber of 'Price on request' in 'price'
print(df['price'].str.contains('Price on request').sum())
print(df['price_cleaned'].isnull().sum())


1015
1015


In [32]:
print(df[(df['price_cleaned'].isna()) & (df['price_s'].isna())].shape[0])

1015
